Kalman Filter, Gerrik
Load the data

In [ ]:
import torch
import numpy as NP
import pandas as PD
import os, scipy
import matplotlib.pyplot as PLT
import Neural_Decoding as ND
from Neural_Decoding.preprocessing_funcs import bin_spikes
from Neural_Decoding.preprocessing_funcs import bin_output
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho
from Neural_Decoding.decoders import KalmanFilterDecoder

T_B=.05 #Size of time bins in seconds

for File_Add in ["../m1_data_raw.mat","../s1_data_raw.mat"]:
    Mat_Dict=scipy.io.loadmat(File_Add,spmatrix=False)
    Keys=list(Mat_Dict.keys())
    """
    spike_times:    When neurons Fired.
    vels       :    X and Y velocities
    vel_times  :    When vel times were taken.
    pos        :    X and Y positions of a cursor
    acc        :    X and Y accuracy of cursor
    """
    
    #Reformat matlab variables to pandas DF.
    DF=dict()
    DF["Vel_x"]=Mat_Dict["vels"][:,0]
    DF["Vel_y"]=Mat_Dict["vels"][:,1]
    DF["Vel_t"]=NP.concat(Mat_Dict["vel_times"])
    DF["Pos_x"]=Mat_Dict["pos"][:,0]
    DF["Pos_y"]=Mat_Dict["pos"][:,1]
    DF["acc_x"]=Mat_Dict["acc"][:,0]
    DF["acc_y"]=Mat_Dict["acc"][:,1]
    Spike_Times=NP.sort(NP.reshape(Mat_Dict["spike_times"],shape=-1))
    PD.DataFrame(DF)

    neural_data=bin_spikes(
        spike_times=DF["Vel_t"],
        dt=T_B,
        wdw_start=DF["Vel_t"][0],
        wds_end=DF["Vel_t"][-1])
    
    vels_binned=bin_output(
        outputs=Mat_Dict["vels"],
        output_times=DF["Vel_t"],
        dt=         T_B,            #Time bin size==0.05 seconds
        wdw_start=  DF["Vel_t"][0], #Start at first sample
        wdw_end=    DF["Vel_t"][-1],#End at last sample
        downsample_factor=1)

__header__
Shape ()
__version__
Shape ()
__globals__
Shape (0,)
spike_times
Shape (164, 1)
vels
Shape (1264999, 2)
vel_times
Shape (1264999, 1)
pos
Shape (1264999, 2)
acc
Shape (1264999, 2)


NameError: name 'neural_recordings' is not defined

<Figure size 1000x500 with 0 Axes>

(2, 1264999)

As can be seen above, both files contain the same variables.

In [ ]:
for File_Add in ["../m1_data_raw.mat","../s1_data_raw.mat"]:
    Mat_Dict=scipy.io.loadmat(File_Add,spmatrix=False)
    print(Mat_Dict.keys())